In [12]:
# Capital Markets Dashboard - Enhanced ETL & AWS Integration
%matplotlib inline
# Step 1: Import Libraries
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output
import requests  # For FRED API integration
import numpy as np
import boto3  # AWS S3 integration
from botocore.exceptions import NoCredentialsError

In [14]:
# Step 2: Set Up FRED API Key and AWS Credentials
FRED_API_KEY = 'YOUR_FRED_API_KEY'  # Replace with your FRED API key
AWS_ACCESS_KEY = 'YOUR_AWS_ACCESS_KEY'  # Replace with your AWS access key
AWS_SECRET_KEY = 'YOUR_AWS_SECRET_KEY'  # Replace with your AWS secret key
S3_BUCKET = 'your-s3-bucket-name'  # Replace with your S3 bucket name

# AWS S3 Client Setup
s3 = boto3.client('s3', aws_access_key_id=AWS_ACCESS_KEY, aws_secret_access_key=AWS_SECRET_KEY)

def upload_to_s3(file_name, bucket, object_name):
    try:
        s3.upload_file(file_name, bucket, object_name)
        print(f"File {file_name} uploaded to S3 bucket {bucket} as {object_name}.")
    except NoCredentialsError:
        print("AWS credentials not available.")

# FRED API Data Fetch
def fetch_fred_data(series_id, start_date, end_date):
    url = f'https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key={FRED_API_KEY}&file_type=json&observation_start={start_date}&observation_end={end_date}'
    response = requests.get(url)
    data = response.json()
    observations = data.get('observations', [])
    dates = [obs['date'] for obs in observations]
    values = [float(obs['value']) if obs['value'] != '.' else None for obs in observations]
    return pd.Series(data=values, index=pd.to_datetime(dates))

In [24]:
# Step 3: Define ETL Functions
# Extract Bond Price Data from Yahoo Finance
def extract_bond_data(ticker, start, end):
    return yf.download(ticker, start=start, end=end)['Close']

# Transform Data
def transform_data(data):
    data = data.dropna()  # remove missing data
    returns = data.pct_change().dropna().cumsum() # Compute cumulative returns
    volatility = data.pct_change().std() * np.sqrt(252)  # Annualized volatility
    sharpe_ratio = (returns.mean() / returns.std()) * np.sqrt(252)  # Simplified Sharpe Ratio
    return data, returns, volatility, sharpe_ratio

# Load Data (Visualization and Reporting)
def load_and_visualize(bond_data, returns, volatility, sharpe_ratio, yield_data, bond_name):
    # Save data to CSV
    bond_data.to_csv(f'{bond_name}_price_data.csv')
    returns.to_csv(f'{bond_name}_returns_data.csv')
    if yield_data is not None:
        yield_data.to_csv(f'{bond_name}_yield_data.csv')
    
    # Upload to AWS S3
    upload_to_s3(f'{bond_name}_price_data.csv', S3_BUCKET, f'data/{bond_name}_price_data.csv')
    upload_to_s3(f'{bond_name}_returns_data.csv', S3_BUCKET, f'data/{bond_name}_returns_data.csv')
    if yield_data is not None:
        upload_to_s3(f'{bond_name}_yield_data.csv', S3_BUCKET, f'data/{bond_name}_yield_data.csv')
    
    # Visualization
    plt.figure(figsize=(10, 5))
    plt.plot(bond_data, label=f'{bond_name} Price')
    plt.title(f'{bond_name} Price Over Time')
    plt.xlabel('Date')
    plt.ylabel('Adjusted Close Price ($)')
    plt.legend()
    plt.grid(True)
    plt.show()

    
    plt.figure(figsize=(10, 5))
    plt.plot(returns, label=f'{bond_name} Cumulative Returns', color='purple')
    plt.title(f'Cumulative Returns for {bond_name}')
    plt.xlabel('Date')
    plt.ylabel('Cumulative Return')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    if yield_data is not None:
        plt.figure(figsize=(10, 5))
        plt.plot(yield_data, label=f'{bond_name} Historical Yield', color='green')
        plt.title(f'Historical Yield for {bond_name}')
        plt.xlabel('Date')
        plt.ylabel('Yield (%)')
        plt.legend()
        plt.grid(True)
        plt.show()

    # Print summary metrics
    print(f"Volatility for {bond_name}: {volatility.mean():.2%}")
    print(f"Sharpe Ratio for {bond_name}: {sharpe_ratio.mean():.2f}")

In [26]:
# Step 4: Set Up Interactive Widgets
tickers = ['SHY', 'IEF', 'TLT']  # Bond ETFs with different durations
fred_series = {'SHY': 'DGS1', 'IEF': 'DGS10', 'TLT': 'DGS30'}  # FRED series IDs for yields

bond_selector = widgets.Dropdown(
    options=tickers,
    description='Select Bond:',
    value='SHY'
)

start_date = widgets.DatePicker(
    description='Start Date',
    value=datetime(2023, 1, 1)
)

end_date = widgets.DatePicker(
    description='End Date',
    value=datetime(2024, 1, 1)
)

update_button = widgets.Button(
    description='Update Data',
    button_style='success'
)

output = widgets.Output()

def update_data(b):
    with output:
        clear_output()
        bond = bond_selector.value
        start = start_date.value.strftime('%Y-%m-%d')
        end = end_date.value.strftime('%Y-%m-%d')
        
        # ETL Process
        raw_data = extract_bond_data(bond, start, end)
        transformed_data, returns, volatility, sharpe = transform_data(raw_data)
        
        # Fetch actual yield data from FRED
        yield_series_id = fred_series.get(bond)
        yield_data = fetch_fred_data(yield_series_id, start, end) if yield_series_id else None
        
        # Load and visualize data
        load_and_visualize(transformed_data, returns, volatility, sharpe, yield_data, bond)

# Attach update function to button
update_button.on_click(update_data)

In [28]:
# Step 5: Display Widgets and Output
display(bond_selector, start_date, end_date, update_button, output)

Dropdown(description='Select Bond:', options=('SHY', 'IEF', 'TLT'), value='SHY')

DatePicker(value=datetime.datetime(2023, 1, 1, 0, 0), description='Start Date', step=1)

DatePicker(value=datetime.datetime(2024, 1, 1, 0, 0), description='End Date', step=1)

Button(button_style='success', description='Update Data', style=ButtonStyle())

Output()